# Decoradores

Los objetivos de aprendizaje son:

1. Características especiales de Funciones
    - First-Class Objects
    - Anidación de funciones
    - Regresar función desde otra función
2. Decoradores Simples
    - Syntaxis
    - Decorar Funciones con argumentos
    - Devolver valores de funciones decoradas
3. Ejemplos reales
    - Timing
    - Debugging
    
    
## Características especiales de Funciones 

Repasemos la idea de una función en Python

In [ ]:
def add_one(numero: int) -> int:
    return numero + 1

add_one(5)

### First-Class Objects

En Python, las funciones están en la categoría `First-Class Objects`. Esto significa que las funciones se pueden pasar y usar como argumentos, como cualquier otro objeto (`string`, `int`, `float`, `list`, etc.). 

Por ejemplo:

In [ ]:
def buenos_dias(nombre: str) -> str:
    return f"Buenos dias {nombre}"

def buenas_tardes(nombre: str) -> str:
    return f"Buenas tardes {nombre}"
    

Ahora definamos una función que toma como argumento otra función.

In [ ]:
from typing import Callable

def saludo(func: Callable)->str:
    return func("Heber")

In [ ]:
saludo(buenos_dias)

`buenos_dias()` y `buenas_tardes()` son funciones regulares que esperan un string. Sin embargo, la función `saludo()` espera una función como argumento. 

### Anidación de funciones

Ya vimos que es posible declarar funciones dentro de otras funciones:

In [ ]:
def padre():
    print("Imprimiendo desde la función padre")

    def primer_hijo():
        print("Imprimiendo desde la función primer_hijo")
    
    def segundo_hijo():
        print("Imprimiendo desde la función segundo_hijo")

    segundo_hijo()
    primer_hijo()

In [ ]:
padre()

>**NOTA**: El orden en que las funciones de declaran no debe ser el mismo en el que se llaman.

In [ ]:
segundo_hijo()

>**Nota**: La función `segundo_hijo()` no existe en el `namespace` global, sólo en el `local` de la función `padre()`

### Regresar función desde otra función

Python también le permite usar funciones como valores de retorno:

In [ ]:
def padre(num: int) -> Callable:
    

    def primer_hijo() -> str:
        return "Heber"

    def segundo_hijo() -> str:
        return "Antonio"
        
    if num == 1:
        return primer_hijo
    else:
        return segundo_hijo

>**Nota**: Estamos deolviendo las funciones sin los paréntesis, es decir sin llamarlas.

In [ ]:
primero = padre(1)
primero()

## Decoradores Simples

Comencemos con un ejemplo:

In [ ]:
def decorador(func_decorar: Callable) -> Callable:
    
    def wrapper() -> None:
        
        func_decorar()
        print("Me han decorado")
    
    return wrapper

def func() -> None:
    print("Soy la función a decorar")



In [ ]:
func()

In [ ]:
func = decorador(func_decorar=func)
func()

> **De manera sencilla**: Hemos envuelto la función `func` dentro de la función `wrapper` al pasarla como argumento a la función `decorador`.

### Sintaxis

La forma en que decoramos `func()` es un poco "rara". En primer lugar, terminamos escribiendo el nombre `func` varias veces. Además, la decoración queda un poco escondida.

Python permite usar decoradores de una manera más simple con el símbolo `@`:

In [ ]:
def decorador(func_decorar: Callable) -> Callable:
    
    def wrapper() -> None:
        
        func_decorar()
        print("Me han decorado")
    
    return wrapper

@decorador
def func() -> None:
    print("Soy la función a decorar")


In [ ]:
func()

Veamos otro ejemplo sencillo

In [ ]:
def repetir(func: Callable) -> Callable:
    def wrapper():
        func()
        func()
    return wrapper

In [ ]:
@repetir
def saludar() -> None:
    print("hola")

In [ ]:
saludar()

## Decorar Funciones con argumentos

Supongamos que tenemos una función que acepta algunos argumentos. ¿Todavía podríamos decorarla?

In [ ]:
@repetir
def saludar(nombre: str) -> None:
    print(f"hola {nombre}")

In [ ]:
saludar(nombre="Heber")

El problema es que la función interna `wrapper()` no toma ningún argumento, pero hemos pasado `nombre="Heber"`. 

La solución es usar `*args` y `**kwargs` en la función `wrapper()`. para que acepte un número arbitrario de argumentos posicionales y del tipo *keyword*:

In [ ]:
def repetir(func: Callable) -> Callable:
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper

@repetir
def saludar(nombre: str) -> str:
    print(f"hola {nombre}")
    
saludar(nombre="Heber")

In [ ]:
@repetir
def saludar_sin_nombre() -> str:
    print("hola")

saludar_sin_nombre()

### Devolver valores de funciones decoradas

¿Qué sucede con el valor de retorno de las funciones decoradas?

In [ ]:
@repetir
def saludos_con_return(nombre: str) -> str:
    print("Creando saludo")
    return f"Hola {nombre}"

saludo_a_heber = saludos_con_return(nombre="Heber")

saludo_a_heber

In [ ]:
print(saludo_a_heber)

El decorador se comió el valor de retorno de la función.

Debido a que `wrapper()` no devuelve explícitamente un valor, la llamada `saludos_con_return(nombre="Heber")` terminó devolviendo `None`.

Para solucionarlo podemos:

In [ ]:
def repetir(func: Callable) -> Callable:
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper

@repetir
def saludos_con_return(nombre: str) -> str:
    print("Creando saludo")
    return f"Hola {nombre}"

saludo_a_heber = saludos_con_return(nombre="Heber")

saludo_a_heber

> **Nota**: La función `saludos_con_return` termina siendo en realidad la función `wrapper`.

In [ ]:
print.__name__

In [ ]:
def saludos_con_return(nombre: str) -> str:
    print("Creando saludo")
    return f"Hola {nombre}"

saludos_con_return.__name__

In [ ]:
@repetir
def saludos_con_return(nombre: str) -> str:
    """DocSrint"""
    print("Creando saludo")
    return f"Hola {nombre}"

saludos_con_return

## Ejemplos reales

Veamos algunos ejemplos más útiles de decoradores.

### Timing

Comencemos por crear un decorador `@timer`. Medirá el tiempo que tarda una función en ejecutarse e imprimirá la duración en la consola:

In [ ]:
import functools
import time

def timer(func: Callable) -> Callable:

    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      
        run_time = end_time - start_time    
        print(f"Finalizando {func.__name__!r} en {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def elevar_al_cuadrado(num_times: int)->None:
    """Docstring"""
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [ ]:
elevar_al_cuadrado(100)

>**Nota**: `functools` es un módulo estándar de Python para funciones de orden superior (funciones que actúan sobre otras funciones o las devuelven). `wraps()` es un decorador que se aplica a la función "envoltura" de un decorador. Actualiza la función contenedora para que parezca la función original

In [ ]:
elevar_al_cuadrado

### Debugging

El siguiente decorador `@debug` imprimirá los argumentos con los que se llama a una función, así como su valor de retorno cada vez que se llama a la función:

In [ ]:
def debug(func: Callable) -> Callable:
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  
        signature = ", ".join(args_repr + kwargs_repr)           
        print(f"LLamando {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"La función {func.__name__!r} regresa {value!r}")           
        return value
    return wrapper_debug

In [ ]:
from typing import Optional

@debug
def saludar(nombre: str, edad: Optional[int] = None) -> str:
    if edad is None:
        return f"Hola {nombre}"
    else:
        return f"Hola {nombre} tu edad es {edad}"

In [ ]:
saludar("Heber")

In [ ]:
saludar("Heber", 30)